In [5]:
import duckdb
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

conn = duckdb.connect(duckdb_path)

with open("sql/load_sakila.sql") as f:
    conn.sql(f.read())

In [7]:
## a) Movies longer than 3 hours

long_movies = conn.sql("""
    SELECT title, length
    FROM film
    WHERE length > 180
    """).df()
##has 38 entries, but doin head (top5) for a shorter notebook for the assignment
long_movies.head() 

,title,length
0,ANALYZE HOOSIERS,181
1,BAKED CLEOPATRA,182
2,CATCH AMISTAD,183
3,CHICAGO NORTH,185
4,CONSPIRACY SPIRIT,184


In [8]:
## b) Movies with "love" in title

## used LLM to find the correct regex match. was fooling around on regex101 website but gave up after an hour.

love_movies = conn.sql("""
    SELECT title, rating, length, description
FROM film
WHERE REGEXP_MATCHES(LOWER(title), '\\blove\\b')
""").df()

love_movies

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
3,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
4,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...


In [9]:
## c) Descriptive statistics on movie length

## TODO: unsure if this is the actual thing we are looking for.

stats = conn.sql("""
    SELECT
        MIN(length) AS shortest,
        AVG(length) AS average,
        MEDIAN(length) AS median,
        MAX(length) AS longest
    FROM film
""").df()

stats

,shortest,average,median,longest
0,46,115.272,114.0,185


In [10]:
## d) 10 most expensive movies per day

expensive_per_day = conn.sql("""
    SELECT
        title,
        rental_rate / rental_duration AS cost_per_day
    FROM film
    ORDER BY cost_per_day DESC
    LIMIT 10
""").df()

expensive_per_day

,title,cost_per_day
0,AMERICAN CIRCUS,1.663333
1,BACKLASH UNDEFEATED,1.663333
2,BILKO ANONYMOUS,1.663333
3,BEAST HUNCHBACK,1.663333
4,CARIBBEAN LIBERTY,1.663333
5,AUTUMN CROW,1.663333
6,CASPER DRAGONFLY,1.663333
7,ACE GOLDFINGER,1.663333
8,BEHAVIOR RUNAWAY,1.663333
9,CASUALTIES ENCINO,1.663333


In [11]:
conn.close()